In [ ]:
!pip install -qU pyngrok
from pyngrok import ngrok
my_auth_token = "2zMjkctLXz9zFtTwmRMbchy3ABW_2i2Cd3v1VisvKdtDbu5X3"
ngrok.set_auth_token(my_auth_token)
public_url = ngrok.connect(7860)
print("Public URL:", public_url)

In [ ]:
from fastapi import FastAPI, Form, UploadFile, File
from pydantic import BaseModel
from fastapi.responses import JSONResponse
from fastapi.staticfiles import StaticFiles
import uuid, os, torch, io, traceback
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
import nest_asyncio
import uvicorn

app = FastAPI()

OUTPUT_DIR = "stable_diffusion_output"
EDITED_DIR = "edited_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(EDITED_DIR, exist_ok=True)

app.mount("/generated", StaticFiles(directory=OUTPUT_DIR), name="generated")
app.mount("/edited", StaticFiles(directory=EDITED_DIR), name="edited")

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to(device)

pipe2 = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

class Prompt(BaseModel):
    text: str

@app.post("/generate_image")
def generate_image(prompt: Prompt):
    try:
        result = pipe(prompt.text)
        image = result.images[0]

        filename = f"{uuid.uuid4().hex}.png"
        file_path = os.path.join(OUTPUT_DIR, filename)
        image.save(file_path)

        return {"url": f"/generated/{filename}"}

    except Exception as e:
        traceback.print_exc()
        return JSONResponse(content={"error": str(e)}, status_code=500)

@app.post("/edit_image")
async def edit_image(prompt: str = Form(...), image: UploadFile = File(...)):
    try:
        img = Image.open(image.file).convert("RGB").resize((512, 512))
        result = pipe2(prompt=prompt, image=img, strength=0.75, guidance_scale=7.5)
        edited = result.images[0]

        filename = f"{uuid.uuid4().hex}.png"
        path = os.path.join(EDITED_DIR, filename)
        edited.save(path)

        return {"url": f"/edited/{filename}"}

    except Exception as e:
        traceback.print_exc()
        return JSONResponse(content={"error": str(e)}, status_code=500)
